# [admin] Images command


The `images` command in `admin` scope could help you manage images.


## Setup PrimeHub Python SDK


In [2]:
from primehub import PrimeHub, PrimeHubConfig
ph = PrimeHub(PrimeHubConfig())

if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
else:
    print("PrimeHub Python SDK couldn't get the group information, follow the 00-getting-started.ipynb to complete it")

PrimeHub Python SDK setup successfully


## Help documentation

In [3]:
help(ph.admin.images)

Help on AdminImages in module primehub.admin_images object:

class AdminImages(primehub.Helpful, primehub.Module)
 |  AdminImages(primehub: primehub.PrimeHub, **kwargs)
 |  
 |  Method resolution order:
 |      AdminImages
 |      primehub.Helpful
 |      primehub.Module
 |      primehub.HTTPSupport
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  add_group(self, id: str, group_id)
 |      Add group connection to an image by id.
 |      
 |      :type id: str
 |      :param id: the id of an image
 |      :type group_id: str
 |      :param group_id: group id
 |      
 |      :rtype dict
 |      :return an image with groups only
 |  
 |  create(self, config)
 |      Create an image
 |      
 |      :type config: dict
 |      :param config: The configurations for creating an image
 |      
 |      :rtype dict
 |      :return The image
 |  
 |  delete(self, id)
 |      Delete an image by id
 |      
 |      :type id: str
 |      :param id: the id of an image
 |      
 |     

## Images management

In [1]:
!primehub admin images help

Usage: 
  primehub admin images <command>

Manage images

Available Commands:
  add-group            Add group connection to an image by id
  create               Create an image
  delete               Delete an image by id
  get                  Get an image by id
  list                 List images
  list-groups          List groups of an image by id
  remove-group         Remove group connection from an image by id
  update               Update the image

Options:
  -h, --help           Show the help

Global Options:
  --config CONFIG      Change the path of the config file (Default: ~/.primehub/config.json)
  --endpoint ENDPOINT  Override the GraphQL API endpoint
  --token TOKEN        Override the API Token
  --group GROUP        Override the current group
  --json               Output the json format (output human-friendly format by default)


### Fields for creating or updating

| field | required | type | description |
| --- | --- | --- | --- |
| name | required | string | it should be a valid resource name for kubernetes. `name` will be ignored when updating |
| displayName | optional | string | |
| description | optional | string | |
| type | optional | string | one of ['cpu', 'gpu', 'both'], default value: 'both' |
| global | optional | boolean |  |
| groups | optional | object | |
| url | optional | string | container image url |
| urlForGpu | optional | string | container image url for GPU optimized |
| imageSpec | optional | object | the specification for customization |
| useImagePullSecret | optional | string | the id of a secret |

*Note: imageSpec cannot use with url and urlForGpu*


## Examples

You could find [more examples on our github](https://github.com/InfuseAI/primehub-python-sdk/blob/main/docs/CLI/admin/images.md).

In [4]:
import pandas as pd

#### CRUD: Jupyter environment images

In [5]:
# Create an image
config = {
  "name": "image-by-sdk",
  "displayName": "Learning how to create an image from SDK",
  "description": "base-notebook with python 3.7",
  "type": "both",
  "url": "infuseai/docker-stacks:base-notebook-63fdf50a",
  "urlForGpu": "infuseai/docker-stacks:base-notebook-63fdf50a-gpu",
  "global": True
}

data = ph.admin.images.create(config)

In [8]:
# List images
images_df = pd.DataFrame(ph.admin.images.list())
images_df[images_df['name'] == config['name']]

,id,name,displayName,description,type,isReady
8,image-by-sdk,image-by-sdk,Learning how to create an image from SDK,base-notebook with python 3.7,both,True


In [6]:
# Get the image details
ph.admin.images.get(data['id'])

{'id': 'image-by-sdk',
 'name': 'image-by-sdk',
 'displayName': 'Learning how to create an image from SDK',
 'description': 'base-notebook with python 3.7',
 'type': 'both',
 'url': 'infuseai/docker-stacks:base-notebook-63fdf50a',
 'urlForGpu': 'infuseai/docker-stacks:base-notebook-63fdf50a-gpu',
 'useImagePullSecret': '',
 'global': True,
 'groups': [],
 'isReady': True,
 'imageSpec': None,
 'jobStatus': None,
 'logEndpoint': 'https://c.demo.primehub.io/api/logs/images/image-by-sdk/job'}

In [10]:
# Update the image config setting.
update_config = {'global': False}
data = ph.admin.images.update(data['id'], update_config)
ph.admin.images.get(data['id'])

{'id': 'image-by-sdk',
 'name': 'image-by-sdk',
 'displayName': 'Learning how to create an image from SDK',
 'description': 'base-notebook with python 3.7',
 'type': 'both',
 'url': 'infuseai/docker-stacks:base-notebook-63fdf50a',
 'urlForGpu': 'infuseai/docker-stacks:base-notebook-63fdf50a-gpu',
 'useImagePullSecret': None,
 'global': False,
 'groups': [],
 'isReady': True,
 'imageSpec': None,
 'jobStatus': None,
 'logEndpoint': 'https://c.demo.primehub.io/api/logs/images/image-by-sdk/job'}

In [11]:
# Delete the image
ph.admin.images.delete(data['id'])

{'id': 'image-by-sdk'}

#### CRUD: image <-> Group

In [12]:
# Create an image
config = {
  "name": "image-by-sdk-group",
  "displayName": "Learning how to create an image from SDK",
  "description": "base-notebook with python 3.7",
  "type": "both",
  "url": "infuseai/docker-stacks:base-notebook-63fdf50a",
  "urlForGpu": "infuseai/docker-stacks:base-notebook-63fdf50a-gpu",
  "global": False
}

data = ph.admin.images.create(config)

In [13]:
# Show the image group list

images_df = pd.DataFrame(ph.admin.images.list_groups(data["id"]))
images_df

""


In [14]:
# Add the image into group
groups_df = pd.DataFrame(ph.admin.groups.list())
groups_id = groups_df["id"][0]
ph.admin.images.add_group(data["id"], groups_id)

In [15]:
# Show the image group list

images_df = pd.DataFrame(ph.admin.images.list_groups(data["id"]))
images_df

,id,name,displayName
0,71ac1e32-65fa-4e8e-a735-ba282e3149b1,AnotherTestGroup,Another Test Group


In [16]:
# Remove the image out of group
ph.admin.images.remove_group(data["id"], groups_id)

In [17]:
# Show the image group list

images_df = pd.DataFrame(ph.admin.images.list_groups(data["id"]))
images_df

""


In [18]:
# Delete the image
ph.admin.images.delete(data['id'])

{'id': 'image-by-sdk-group'}